In [1]:
from helpers.data_helpers import get_availability, get_hours, get_shift_data
from helpers.scheduling_helpers import find_match, report_open_slots, print_schedule, report_ta_under_schedules

# Get TA Hours and availability from data files

In [2]:
[hours, seniority] = get_hours("data/TA-hours.csv")
[tutoring_scores, tutoring_avail] = get_availability("data/tutoring.csv")
[oh_scores, oh_avail] = get_availability("data/oh.csv")
tutoring_slots = get_shift_data("shift_data/tutoring_shifts.csv")
oh_slots = get_shift_data("shift_data/oh_shifts.csv")
shift_lead_slots = get_shift_data("shift_data/oh_shift_leads.csv")

In [3]:
# If TA has 0 hours availability, remove them from the dict.
remove = []
for ta in hours:
    if hours[ta] == 0:
        remove.append(ta)

for member in remove:
    hours.pop(member)
    seniority.pop(member)
    print(f"{member} fully scheduled")

### Make sure that names in WhenIsGoodData match names in TA Hours.

In [4]:
names = tutoring_avail.keys()
for name in names:
    if not name in hours.keys():
        print(f"{name} missing from TA hours")     

In [5]:
names = oh_avail.keys()
for name in names:
    if not name in hours.keys():
        print(f"{name} missing from TA hours")

### Make sure all TAs gave availability

In [18]:
names = hours.keys()
for name in names: 
    if not name in oh_avail.keys():
        print(f"{name} missing availability")

anjali shah
austin hill
benjamin eldridge
krisha avula
mitchell anderson
oliver blomfield


In [7]:
# # Used this to compare total OH scheduled hours with TA availability
# total_hours = 0
# for key in oh_slots:
#     total_hours += oh_slots[key]
# for key in shift_lead_slots:
#     total_hours += 1
# for key in tutoring_slots:
#     total_hours += tutoring_slots[key]
# print(total_hours)

# ta_hours = 0
# for key in hours:
#     ta_hours += hours[key]
# print(ta_hours)

# Order Team Members by Seniority and break into two groups

In [8]:
# order TAs by seniority: highest to lowest
ta_seniority = {k:v for k, v in sorted(seniority.items(), reverse=True, key=lambda item: item[1])}

senior_members = []
junior_members = []
for ta in ta_seniority:
    if ta_seniority[ta] > 1:
        senior_members.append(ta)
    else:
        junior_members.append(ta)

# Functions to Assign Shifts

In [9]:
def assign_senior_members(schedule: dict, shifts: list, availability: dict, slots: dict):
    for shift in shifts:
        if slots[shift] > 0:
            # exclude already scheduled tas for that shift
            exclude = schedule[shift]
            team_member = find_match(senior_members, shift, availability, exclude)
            if team_member != "None":
                # reduce amount of schedulable hours for that member
                hours[team_member] -= 1
                # add member to schedule
                schedule[shift].append(team_member)
                # if team member has reached max hours, remove from list
                if hours[team_member] == 0:
                    senior_members.remove(team_member)
                # reduce slots for that shift
                slots[shift] -= 1
            else:
                print(f"No senior member found for shift {shift}")


In [10]:
def assign_junior_members(schedule: dict, shifts: list, availability: dict, slots: dict):
    for shift in shifts:
        if slots[shift] > 0:
            # exclude already scheduled tas for that shift
            exclude = schedule[shift]
            team_member = find_match(junior_members, shift, availability, exclude)
            if team_member != "None":
                # reduce amount of schedulable hours for that member
                hours[team_member] -= 1
                # add member to schedule
                schedule[shift].append(team_member)
                # if team member has reached max hours, remove from list
                if hours[team_member] == 0:
                    junior_members.remove(team_member)
                # reduce slots for that shift
                slots[shift] -= 1
            else:
                print(f"No junior member found for shift {shift}")


# Tutoring Shifts

In [11]:
# order tutoring shifts by lowest "availability scores" given by WhenIsGood
shifts = [k for k, v in sorted(tutoring_scores.items(), key=lambda item: item[1])]

# Assign senior members to shifts
tutoring_schedule = {}
for shift in tutoring_slots:
    tutoring_schedule[shift] = []


# Assign senior members first
assign_senior_members(tutoring_schedule, shifts, tutoring_avail, tutoring_slots)
assign_senior_members(tutoring_schedule, shifts, tutoring_avail, tutoring_slots)
assign_junior_members(tutoring_schedule, shifts, tutoring_avail, tutoring_slots)
assign_junior_members(tutoring_schedule, shifts, tutoring_avail, tutoring_slots)
assign_junior_members(tutoring_schedule, shifts, tutoring_avail, tutoring_slots)


No senior member found for shift Wed 17:00
No senior member found for shift Wed 18:00
No junior member found for shift Thu 17:00
No junior member found for shift Thu 18:00
No junior member found for shift Thu 17:00
No junior member found for shift Thu 18:00


## Tutoring Schedule

In [12]:
print_schedule(tutoring_schedule)
report_open_slots(tutoring_slots)


Mon 17:00:
shaina patel
lynn lee
jimena rivera
samy vipin

Mon 18:00:
shaina patel
aj valentino
yashu singhai
aknazar janibek

Wed 17:00:
lynn lee
jimena rivera
samy vipin
aknazar janibek

Wed 18:00:
aj valentino
yashu singhai
jimena rivera
aknazar janibek

Thu 17:00:
aj valentino
lynn lee
samy vipin

Thu 18:00:
aj valentino
shaina patel
yashu singhai
1 people missing from Thu 17:00
1 people missing from Thu 18:00


# Shift Leads

In [13]:
# order tutoring shifts by lowest "availability scores" given by WhenIsGood
shifts = [k for k, v in sorted(oh_scores.items(), key=lambda item: item[1])]

# Assign senior members to shifts
shift_lead_schedule = {}
for shift in shift_lead_slots:
    shift_lead_schedule[shift] = []


# Assign senior members first
assign_senior_members(shift_lead_schedule, shifts, oh_avail, shift_lead_slots)

# Shift Lead Schedule

In [14]:
print_schedule(shift_lead_schedule)
report_open_slots(shift_lead_slots)


Mon 11:00:
jayden lim

Mon 12:00:
jayden lim

Mon 13:00:
sophie jiang

Mon 14:00:
sarah chocron

Mon 15:00:
audrey salmon

Mon 16:00:
audrey salmon

Tue 11:00:
audrey salmon

Tue 12:00:
aj valentino

Tue 13:00:
jayden lim

Tue 14:00:
bridget lin

Tue 15:00:
bridget lin

Tue 16:00:
vincent li

Wed 11:00:
jayden lim

Wed 12:00:
jayden lim

Wed 13:00:
grace ku

Wed 14:00:
vincent li

Wed 15:00:
grace ku

Wed 16:00:
grace ku

Thu 11:00:
jayden lim

Thu 12:00:
aj valentino

Thu 13:00:
jayden lim

Thu 14:00:
lynn lee

Thu 15:00:
bridget lin

Thu 16:00:
aj valentino

Fri 11:00:
jayden lim

Fri 12:00:
kene ochuba

Fri 13:00:
kene ochuba

Fri 14:00:
shefali pai

Fri 15:00:
carmine anderson

Fri 16:00:
kene ochuba

Sun 13:00:
sarah chocron

Sun 14:00:
sarah chocron

Sun 15:00:
sarah chocron

Sun 16:00:
sarah chocron


# Office Hours

In [15]:
# order tutoring shifts by lowest "availability scores" given by WhenIsGood
shifts = [k for k, v in sorted(oh_scores.items(), key=lambda item: item[1])]
# prioritize certain shifts that didn't get scheduled
prioritize = ["Tue 16:00", "Tue 13:00"]
for shift in shifts:
    if not shift in prioritize:
        prioritize.append(shift)


# Assign senior members to shifts
oh_schedule = {}
for shift in oh_slots:
    oh_schedule[shift] = []


# Assign senior members first
for _ in range(4):
    assign_senior_members(oh_schedule, prioritize, oh_avail, oh_slots)
for _ in range(4):
    assign_junior_members(oh_schedule, prioritize, oh_avail, oh_slots)

No senior member found for shift Thu 12:00
No senior member found for shift Tue 16:00
No senior member found for shift Tue 15:00
No senior member found for shift Thu 15:00
No senior member found for shift Thu 13:00
No senior member found for shift Wed 15:00
No senior member found for shift Tue 12:00
No senior member found for shift Wed 16:00
No senior member found for shift Mon 11:00
No senior member found for shift Thu 12:00
No senior member found for shift Thu 16:00
No senior member found for shift Mon 13:00
No senior member found for shift Mon 12:00
No senior member found for shift Fri 15:00
No senior member found for shift Fri 13:00
No senior member found for shift Fri 16:00
No senior member found for shift Sun 16:00
No senior member found for shift Sun 15:00
No senior member found for shift Fri 14:00
No senior member found for shift Sun 13:00
No senior member found for shift Sun 14:00
No senior member found for shift Tue 16:00
No senior member found for shift Tue 13:00
No senior m

# Office Hours Schedule

In [16]:
print_schedule(oh_schedule)
report_open_slots(oh_slots)


Mon 11:00:
coralee rogers-vickers
gabriela barros
lizzie coats
julia guzzo
tracy dang

Mon 12:00:
sophie jiang
catherine huang
julia guzzo
luke aiello
conor jones

Mon 13:00:
sophie jiang
catherine huang
onyi igwe
luke aiello
yashu singhai

Mon 14:00:
ella gonzales
upasana lamsal
onyi igwe
luke aiello
wei jiang
yashu singhai

Mon 15:00:
sarah chocron
miguel villasenor
aknazar janibek
rebecca henriques
charlotte tsui
ivy nangalia

Mon 16:00:
miguel villasenor
daisy azagba
aknazar janibek
narasimha cittarusu
rebecca henriques
terrence hampton

Tue 11:00:
viktorya hunanyan
dylan nicks
julia guzzo
katie brown
terrence hampton
sanjana gopalswamy

Tue 12:00:
jack coury
lizzie coats
onyi igwe
ryan krasinski
grace odondi

Tue 13:00:
lynn lee
shaina patel
lucas siegel
mariana rodriguez-pacheco
ryan krasinski
somer lillard

Tue 14:00:
lynn lee
daisy azagba
adrian lanier
julia guzzo
somer lillard
michelle kieu

Tue 15:00:
alex xiao
adrian lanier
lucas siegel
somer lillard
michelle kieu

Tue 16:0

In [17]:
report_ta_under_schedules(hours)

7 hours not scheduled for anjali shah
2 hours not scheduled for austin hill
7 hours not scheduled for benjamin eldridge
7 hours not scheduled for caitlin estrada
2 hours not scheduled for ivy nangalia
7 hours not scheduled for krisha avula
8 hours not scheduled for mitchell anderson
4 hours not scheduled for oliver blomfield
2 hours not scheduled for prachi joshi
4 hours not scheduled for tobenna okoli
1 hours not scheduled for yashu singhai
1 hours not scheduled for charlotte tsui
